<a href="https://colab.research.google.com/github/erickfaria/ibge/blob/main/populacao_ibge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.0 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import zipfile
from ftplib import FTP
import datetime
from dateutil.relativedelta import relativedelta
import tqdm
import pandas as pd
import numpy as np

In [ ]:
year = datetime.date.today().year - 1
last_month = datetime.date.today().month - 1
month = '{:02d}'.format(last_month)

In [ ]:
pop = f'estimativa_dou_{year}.xls'

ftp = FTP(f'ftp.ibge.gov.br')
ftp.login()
ftp.cwd(f'Estimativas_de_Populacao/Estimativas_{year}')

'250 Directory successfully changed.'

In [ ]:
with open(f'{pop}', 'wb') as fp:
  ftp.retrbinary(f'RETR {pop}', fp.write)

# Limpa o Dado

In [ ]:
df = pd.read_excel('/content/estimativa_dou_2021.xls', 'Municípios', skiprows=1,
                   nrows=5570, header=0, 
                   names=['nom_uf', 'cod_uf', 'cod_mun', 'nom_mun','pop_mun'],
                   dtype={'UF':str,	'cod_uf':str,	'cod_mun':str,
                           'nom_mun':str, 'pop_mun':str})

In [ ]:
df['cod_mun'] = df['cod_uf'] + df['cod_mun']

In [ ]:
df['pop_mun'] = df['pop_mun'].str.replace(r"\(.*\)","")
df['pop_mun'] = df['pop_mun'].str.replace(r".","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
df = df.astype({'nom_uf': str,
                'cod_uf': 'int32',
                'cod_mun': 'int32',
                'nom_mun': str,
                'pop_mun': 'int32'})

# Salva os Dados

In [ ]:
df.to_parquet(f'/content/pop_ibge_{year}.parquet.gzip',
              compression='gzip', index=False)